In [2]:
import os
import numpy as np

In [ ]:
def block_bootstrap(data, block_size, n_samples = 1000):
    n = len(data)
    blocks = [data[i:i+block_size] for i in range(0, n - block_size + 1)]
    n_blocks = int(np.ceil(n/block_size))
    bootstrap_samples = []
    for _ in range(n_samples):
        sample = np.concatenate([blocks[np.random.randint(0, len(blocks))] for _ in range(n_blocks)])
        bootstrap_samples.append(sample[:n]) 
    return np.array(bootstrap_samples)

In [3]:
path_in = '../../data'
path_out = '../../outputs/bias_test'

In [ ]:
season = 'djf'  # djf jja

models = ['cfsv2', 'seas5']

seas5_arr = np.nanmean(np.load(os.path.join(path_in, f'seas5_{season}.npz'))['WS10M'], axis=1)
cfsv2_arr = np.nanmean(np.load(os.path.join(path_in, f'cfsv2_{season}.npz'))['WS10M'], axis=1)
merra2_arr = np.load(os.path.join(path_in, f'merra2_{season}.npz'))['WS10M']

lon_arr = np.load(os.path.join(path_in, f'cfsv2_{season}.npz'))['longitude']
lat_arr = np.load(os.path.join(path_in, f'cfsv2_{season}.npz'))['latitude']

data_cfsv2 = merra2_arr - cfsv2_arr
data_seas5 = merra2_arr - seas5_arr

(24, 80, 80)

In [ ]:
cfsv2_boot = np.apply_along_axis(func1d = block_bootstrap, axis = 0, arr = data_cfsv2, block_size=3)
seas5_boot = np.apply_along_axis(func1d = block_bootstrap, axis = 0, arr = data_seas5, block_size=3)

# Calculate bias for each resample
cfsv2_boot_bias = np.nanmean(cfsv2_boot, axis=1)
seas5_boot_bias = np.nanmean(seas5_boot, axis=1)

# Significance
alpha = 5
# Two tail test
ci_lower_cfsv2, ci_upper_cfsv2 = np.percentile(cfsv2_boot_bias, 
                                               [alpha/2, (100-alpha)/2], axis=0)
ci_lower_seas5, ci_upper_seas5 = np.percentile(seas5_boot_bias, 
                                                   [alpha/2, (100-alpha)/2], axis=0)

# False for statistically significant biases and True for not statistically significant biases 
results_cfsv2 = np.zeros(ci_upper_cfsv2.shape)
results_cfsv2[(ci_upper_cfsv2 >= 0) & (ci_lower_cfsv2 <= 0)] = 1
results_cfsv2 = np.array(results_cfsv2, dtype=bool)

results_seas5 = np.zeros(ci_lower_seas5.shape)
results_seas5[(ci_upper_seas5 >= 0) & (ci_lower_seas5 <= 0)] = 1
results_seas5 = np.array(results_seas5, dtype=bool)

In [ ]:
np.savez(os.path.join(path_out, f"bootstrap_bias_cfsv2_{season}.npz"), test_result = results_cfsv2
        lon = lon_arr, lat = lat_arr)
np.savez(os.path.join(path_out, f"bootstrap_bias_seas5_{season}.npz"), test_result = results_seas5,
         lon = lon_arr, lat = lat_arr)